<center><h1>网络安全入侵检测挑战赛</h1></center>

> 报名链接：https://challenge.xfyun.cn/topic/info?type=image-style-transfer&ch=dw24_AtTCK9

# 一、赛题背景

随着互联网技术的飞速发展，网络安全已经成为全球关注的焦点。网络入侵事件频发，给个人隐私、企业数据安全乃至国家安全带来了严重威胁。为了提高网络安全防护能力，有效识别和防御各种网络攻击和入侵行为，本场比赛以“网络安全入侵检测”为主题的技术挑战赛。选手根据训练数据构建模型，预测是否存在网络攻击以及对应的攻击类型。

![img](https://openres.xfyun.cn/xfyundoc/2024-07-17/fc75f492-920c-47db-82ce-e324b2f988fa/1721207416448/140.png)

# 二、赛题任务

为了有效地分析和分类如此大量的文件，赛题提供了已知的网络攻击数据，需要选手预测每个样本数据所属类别，这其中也包含良性样本。这是一个多分类问题，由0到N自然数进行标识。

为了鼓励选手在算法设计和优化上的创新，本场比赛不鼓励过度堆砌模型，对多模型策略将进行限制，要求在预测阶段的模型组合不能超过2个，压缩包总大小不超过1G。

备注：使用多个模型的结果进行简单融合，被认为是多模型策略。

# 三、评审规则

## 1.数据说明

赛题数据由训练集和测试集组成，总数据量超过百万，包含45个字段，其中id为每条样本的唯一标识符，label为样本所属网络攻击类别。为了保证比赛的公平性，将会从中抽取一部分数据作为训练集，一部分作为测试集，同时会对部分字段信息进行脱敏。

| 特征字段 | 字段描述                             |
| -------- | ------------------------------------ |
| id       | 样本id，唯一表示，无重复             |
| feat_n   | 特征字段，进行脱敏处理               |
| label    | 预测目标，网络攻击类别，包含良性流量 |

## 2.评审规则

本次竞赛的评价标准采用准确率指标，最高分为1。计算方法参考https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

评估代码参考：

1 from sklearn.metrics import accuracy_score

2 y_pred = [0, 2, 1, 3]

3 y_true = [0, 1, 2, 3]

4 accuracy_score(y_true, y_pred)

## 3.评测及排行

1）赛事提供下载数据，选手在本地进行算法调试，在比赛页面提交结果。

2）排行按照得分从高到低排序，排行榜将选择团队的历史最优成绩进行排名。

# 四、作品提交

1.文件格式：预测结果文件按照.csv格式提交

2.文件大小：无要求

3.提交次数：每支队伍每天最多3次

4.预测结果文件详细说明：

• a.编码为UTF-8，第一行为表头；

• b.提交前请确保预测结果的格式与sample_submit.csv中的格式一致。具体格式如下：

1 id,label

2 800000,0

3 800001,0

4 800002,0

5 800003,0

6 800004,0

# 五、赛程规则

## 正式赛：7月18日 — 8月20日

初赛截止成绩以团队在初赛时间段内最优成绩为准，具体排名可见初赛榜单。

初赛作品提交截止日期为8月20日17:00；正式赛名次将于结束后15天内公布。

## 长期赛：8月20日 —10月24日

正式赛结束后，将转变为长期赛，供开发者学习实践。本阶段提交后，系统会根据成绩持续更新长期赛榜单，但该阶段榜单不再进行奖励。

# 六、奖项设置

本赛题设立一、二、三等奖，具体详情如下：

一等奖：1支队伍，周赛一等奖证书，奖金：1000元

二等奖：1支队伍，周赛二等奖证书，奖金：800元

三等奖：1支队伍，周赛三等奖证书，奖金：500元

注：

1. 鼓励选手分享参赛心得、参赛技术攻略、大赛相关技术或产品使用体验等文章至组委会邮箱（AICompetition@iflytek.com），有机会获得大赛周边；
2. 赛事规则及奖金发放解释权归科大讯飞所有；以上全部奖金均为税前金额，将由主办方代扣代缴个人所得税。

In [21]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
import lightgbm as lgb

In [6]:
train_data = pd.read_csv('dataset/train_data.csv')
test_data = pd.read_csv('dataset/test_data.csv')

In [9]:
train_data.shape

(800000, 45)

In [10]:
test_data.shape

(200000, 44)

In [8]:
train_data['label'].value_counts()

label
1    266424
2    230767
0    189051
5     40326
3     27876
6     26029
7     12176
4      7351
Name: count, dtype: int64

In [18]:
pred = cross_val_predict(
    LogisticRegression(),
    train_data.iloc[:, 1:-1].head(10000),
    train_data['label'].head(10000)
)

accuracy_score(train_data['label'].head(10000), pred)

0.7789

In [19]:
pred = cross_val_predict(
    DecisionTreeClassifier(),
    train_data.iloc[:, 1:-1].head(10000),
    train_data['label'].head(10000)
)

accuracy_score(train_data['label'].head(10000), pred)

0.9725

In [22]:
model = lgb.LGBMClassifier()
model.fit(train_data.iloc[:, 1:-1], train_data['label'])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.059161 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7233
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 43
[LightGBM] [Info] Start training from score -1.442595
[LightGBM] [Info] Start training from score -1.099523
[LightGBM] [Info] Start training from score -1.243203
[LightGBM] [Info] Start training from score -3.356846
[LightGBM] [Info] Start training from score -4.689775
[LightGBM] [Info] Start training from score -2.987615
[LightGBM] [Info] Start training from score -3.425400
[LightGBM] [Info] Start training from score -4.185145


LGBMClassifier()

In [26]:
pd.DataFrame({
    'id': test_data['id'],
    'label': model.predict(test_data.iloc[:, 1:])
}).to_csv('submit.csv', index=None)